# Building Multi-Agent AI Systems: From Concept to Implementation

## Introduction

AI systems are becoming increasingly complex, tackling problems that require multiple
specialized capabilities working in concert. In this tutorial, we'll build a multi-agent
system where specialized AI agents collaborate to accomplish complex tasks.

We'll explore:
1. Why multi-agent architectures are valuable
2. The challenges of building multi-agent systems
3. How to implement a multi-agent system using mahilo
4. Building a real-world multi-agent system step-by-step

## Why Multi Agent Systems?



Multi-agent systems offer several key advantages:

1. **Specialization**: Different agents can focus on specific tasks, leading to better
   performance in each domain - just like specialists in a team.

   ![](https://miro.medium.com/v2/resize:fit:640/format:webp/0*zf7GL-Dz8JAOB7tf.gif)

2. **Task Decomposition**: Complex problems become manageable when broken down into
   smaller sub-problems assigned to specialized agents.

3. **Parallel Processing**: Multiple agents can work simultaneously on different aspects
   of a problem, increasing efficiency.

4. **Better Accuracy**: Increasing the size of the toolset for a given agent leads to a dip in accuracy as shown by the ReWOO paper.

   ![](https://miro.medium.com/v2/resize:fit:640/format:webp/1*GvD5uvHY0mOPrmpp9Oh6-A.png)

## The Challenges of Building Multi-Agent Systems

Despite their benefits, building effective multi-agent systems presents challenges:

### 1. Communication Complexity

Enabling effective communication between agents requires:

- Designing protocols for agents to request information from each other
- Ensuring messages contain appropriate context
- Tracking conversations and maintaining state across interactions
- Preventing infinite loops and circular references

### 2. Observability Challenges

As your agent system grows, understanding what's happening becomes difficult:

- Tracing requests across multiple agents
- Identifying which agent might be causing issues
- Collecting metrics on agent performance
- Debugging failures in a distributed system

### 3. Human-in-the-Loop Complexity

Integrating human oversight adds another layer of complexity:

- Creating interfaces for humans to monitor agent activities
- Designing effective handoff protocols between AI and humans
- Determining when to escalate decisions to human operators

## mahilo: A Lightweight Control Plane for Multi-Agent Systems

Mahilo is a framework designed to address these challenges while remaining lightweight
and flexible. To understand how mahilo works, let's start by building a simple multi-agent system.

We'll create a team of two agents:

1. A **Research Agent** that can find information
2. A **Writing Agent** that can create content based on that information

First, let's define our research agent. The process for the Writing Agent will be similar and we will not show it for brevity.

### Setup
Before, we begin, let's install the mahilo package.

In [ ]:
!pip install mahilo

Add an OpenAI API key to the environment variables.

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

### Defining the tools and prompt for the research agent

In [2]:
from mahilo import BaseAgent, AgentManager, ServerManager

In [6]:
def simple_search(query) -> str:
    """
    A simple mock search function for demonstration purposes.
    In a real application, this would connect to a search API.
    """
    if "python" in query.lower():
        return "Python is a high-level, interpreted programming language known for its readability."
    elif "javascript" in query.lower():
        return "JavaScript is a scripting language primarily used for web development."
    else:
        return f"Here are some facts about {query}..."

# Define the research agent's tools
research_tools = [
    {
        "tool": {
            "type": "function",
            "function": {
                "name": "search",
                "description": "Search for information on a topic",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query"
                        }
                    },
                    "required": ["query"]
                }
            }
        },
        "function": simple_search,
    }
]

# Define the research agent's prompt
research_agent_prompt = """
You are a research agent that finds information on topics.
When asked about a topic, use your search tool to find relevant information.
Be concise and factual in your responses.
"""

# Then, a concise description that helps other agents understand its role
research_agent_short_description = """
Research agent that can search for information on various topics
"""

### Now, let's create our agents and set up the system:

In [7]:
def setup_simple_system() -> ServerManager:
    # Create the agents
    research_agent = BaseAgent(
        name="ResearchAgent",
        type="research_agent",
        description=research_agent_prompt,
        short_description=research_agent_short_description,
        tools=research_tools,
    )
    
    writing_agent = BaseAgent(
        name="WritingAgent",
        type="writing_agent",
        description="You are a writing agent",
        short_description="A writing agent",
        tools=[],
        can_contact=["research_agent"]  # Specify which agents this agent can contact
    )
    
    # Create an agent manager to manage our team
    team = AgentManager()
    
    # Register the agents with the manager
    team.register_agent(research_agent)
    team.register_agent(writing_agent)
    
    # Activate the agents
    research_agent.activate()
    writing_agent.activate()
    
    # Create a server manager to handle agent communication
    server = ServerManager(team)
    
    return server

In [ ]:
import asyncio
import nest_asyncio

server = setup_simple_system()

# run the server
nest_asyncio.apply()
asyncio.run(server.run())

You can now connect to the server using the mahilo CLI in your terminal.This will open a chat where you can talk to the research agent about your task. The agent can then process your request and reach out to the writing agent as needed to complete the task.

```bash
mahilo connect --agent-name ResearchAgent
```

What's important to notice here
1. **Automatic Communication**: The writing agent can automatically communicate with the
   research agent thanks to mahilo's communication layer.

2. **Clear Separation of Concerns**: Each agent has a specific role and set of capabilities,
   making the system modular and maintainable.

3. **Simple Setup**: Despite being a multi-agent system, the setup code is straightforward
   and easy to understand.

4. **Minimal Boilerplate**: Mahilo handles all the communication infrastructure, so we can
   focus on defining the agents' capabilities and behavior.

In a real application, the server would be running, and you'd interact with it through API
calls or a web interface. Each agent would have more sophisticated tools, and the communication
would happen automatically as needed.


### Built for complex applications

In addition to the features above, mahilo boasts other features that make
it a powerful tool for building multi-agent systems.

1. Built-in Observability

    Mahilo provides out-of-the-box metrics, traces (in OpenTelemetry format), and logs of all
inter-agent communications, making it easy to understand, debug, and optimize your system.
    #  <div style="display: flex; flex-direction: row; justify-content: space-between; gap: 10px;">
    #    <img src="../../assets/metrics.png" alt="metrics" style="max-width: 48%;" />
    #    <img src="../../assets/traces.png" alt="traces" style="max-width: 48%;" />
    #  </div>
2. Human-in-the-Loop First

    Unlike many frameworks where human oversight is an afterthought, mahilo treats humans as
first-class participants. Any agent can have a human counterpart who can monitor, intervene,
and guide the agent as needed.

3. Framework Agnostic

    Mahilo is designed to work with agents from any framework, whether they're built with
LangChain, PydanticAI, or custom implementations. This makes it easy to 
incorporate existing agents or specialized tools into your system.

Watch a demo of a mahilo system in action:

[![](../../assets/mahilo_tut_simple.jpg)](https://x.com/wjayesh/status/1872263352254427458)


Now that we understand the basics, let's move on to a more complex real-world example:
a multi-agent system for filmmaking.

## The Hitchcock Project: A Multi-Agent Filmmaker

Let's examine a more complex example: a multi-agent system for filmmaking
that won the first prize in the [ElevenLabs Worldwide Hackathon](https://hackathon.elevenlabs.io/) for the Bengaluru track. This system has multiple agents that work together to output a film in the end. Here, I'll explain the thought process behind building such a system, and how I went about designing and executing it.

### Step 1: Planning the System Architecture

Before writing any code, it's crucial to plan the overall architecture of your multi-agent system.
For our filmmaker project, we started by:

1. **Identifying the Core Processes**: Analyzing the filmmaking process to identify distinct stages
   that could benefit from specialization

2. **Defining Agent Boundaries**: Determining clear responsibilities for each agent to avoid overlap
   and ensure clear ownership of tasks

3. **Mapping Communication Flows**: Deciding which agents need to communicate with each other
   and what information they need to exchange

4. **Designing Workflow Sequences**: Establishing the correct order of operations to produce
   a coherent final product

![hithcock_arch](../../assets/hitchcock_arch.jpg)

Our analysis led us to a system with four specialized agents:

1. **Script Writer Agent**: Creates the initial movie script based on a prompt
2. **Story Boarder Agent**: Breaks down the script into visual scenes and shots
3. **Director of Photography (DOP) Agent**: Generates images based on storyboard specifications
4. **Audio Agent**: Creates audio content matching the visual scenes

This architecture ensures that each stage of film production has a dedicated expert agent,
with clear workflows between them.

### Step 2: Designing Agent Capabilities

For each agent, we needed to:

1. **Define Core Competencies**: What specific tasks can this agent perform?
2. **Design Appropriate Tools**: What functions should the agent have access to?
3. **Craft Effective Prompts**: How should we instruct the agent to use these tools?
4. **Establish Communication Protocols**: When and how should this agent interact with others?

Let's look at how we designed the Script Writer agent:

In [ ]:
# First, we defined the prompt that establishes the agent's role and behavior
script_writer_prompt = """
You are a creative story writer agent specialized in creating movie scripts. You have access to powerful research tools:

1. Web Search and Browsing:
   - search_information: Search the internet for information
   - visit: Visit and read web pages
   - page_up/page_down: Navigate through long content
   - find/find_next: Search within pages
   - search_archives: Search historical archives

2. Text Analysis:
   - inspect_file_as_text: Analyze and extract information from text content

When given a writing task:
1. First research the historical and cultural context of your setting
2. Study similar movies and their themes for inspiration
3. Use the research to create authentic, well-researched scripts
4. Pay special attention to:
   - Dialogue matching the era and location
   - Historically accurate scene descriptions
   - Cultural authenticity
   - Period-appropriate details

Once you're done with the script generation, inform the story boarder agent to create a storyboard. Always do that, and you can
use the chat_with_agent tool for it.

Wait for specific requests before taking action. Use your research tools to gather information before writing.
"""

# Then, a concise description that helps other agents understand its role
script_writer_short_description = """
Story writer agent for creating movie scripts with deep research capabilities
"""

Now, we implemented key tools that the agent would need to fulfill its role.

> Note: This is a simplified version of the actual implementation

In [ ]:
def get_script_with_research(script_prompt):
    """
    Generate a movie script based on the given prompt with research capabilities.
    
    Args:
        script_prompt (str): Description of the script to generate
        
    Returns:
        str: Generated script text
    """
    # In the real implementation, this would:
    # 1. Use a research agent to gather relevant information
    # 2. Generate a full script using an LLM
    # 3. Save the script to a shared location for other agents to access
    # 4. Extract character information for the storyboard agent
    
    # For demonstration purposes, we'll just return a placeholder
    return f"Generated script based on: {script_prompt}"

The next step is to package the tool in mahilo's expected format. This is just the OpenAI tool definition spec with an additional function field.

In [ ]:
# Package the tool in mahilo's expected format
script_writer_tools = [
    {
        "tool": {
            "type": "function",
            "function": {
                "name": "get_script_with_research",
                "description": "Generate a movie script with research capabilities",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "script_prompt": {
                            "type": "string",
                            "description": "Description of the script to generate (e.g., 'Write a thriller set in 1920s Chicago')"
                        }
                    },
                    "required": ["script_prompt"]
                }
            }
        },
        "function": get_script_with_research,
    }
]